In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time


In [9]:
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=Assistances&agg=Accumulated"
years = list(range(2001, 2022))

In [10]:
for year in years:
    with open("assists_leaders_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)
        

In [11]:
dfs = []
for year in years:
    with open("assists_leaders_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    assists = pd.read_html(str(player_table))[0]
    assists["Year"] = year
    dfs.append(assists)

In [12]:
assists = pd.concat(dfs)
assists

,Rank,Player,Team,GamesG,AssistsAS,Ave.,Per 40 min.40m,Year
0,1,"BENNETT, ELMERBENNETT, E.",BKNTau CeramicaTau Ceramica,15,79,5.27,6.96,2001
1,2,"McDONALD, ARRIELMCDONALD, A.",MACMaccabi EliteMaccabi Elite,20,76,3.80,4.78,2001
2,2,"PAPALOUKAS, THEODOROSPAPALOUKAS, T.",OLYOlympiacosOlympiacos,19,76,4.00,5.94,2001
3,4,"BOOKER, MELVINBOOKER, M.",PESScavolini PesaroScavolini Pesaro,20,75,3.75,4.44,2001
4,5,"EDNEY, TYUSEDNEY, T.",TREBenetton BasketBenetton Basket,19,72,3.79,5.00,2001
...,...,...,...,...,...,...,...,...
45,45,"SHIELDS, SHAVONSHIELDS, S.",AXMAX Armani Exchange MilanAX Armani Exchange ...,24,67,2.79,3.75,2021
46,47,"ULANOVAS, EDGARASULANOVAS, E.",ZALZalgiris KaunasZalgiris Kaunas,32,66,2.06,3.00,2021
47,47,"SPISSU, MARCOSPISSU, M.",UNKUNICS KazanUNICS Kazan,25,66,2.64,7.33,2021
48,49,"IVANOVIC, NIKOLAIVANOVIC, N.",CZVCrvena Zvezda mts BelgradeCrvena Zvezda mts...,27,65,2.41,6.27,2021


In [13]:
engine = create_engine('sqlite:///assists_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [14]:
assists.to_sql(
    name = "assists_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [15]:
assists_select = session.execute('select * from "assists_leaders"')
assists_select.all()

[(0, 1, 'BENNETT, ELMERBENNETT, E.', 'BKNTau CeramicaTau Ceramica', 15, 79, 5.27, 6.96, 2001),
 (1, 2, 'McDONALD, ARRIELMCDONALD, A.', 'MACMaccabi EliteMaccabi Elite', 20, 76, 3.8, 4.78, 2001),
 (2, 2, 'PAPALOUKAS, THEODOROSPAPALOUKAS, T.', 'OLYOlympiacosOlympiacos', 19, 76, 4.0, 5.94, 2001),
 (3, 4, 'BOOKER, MELVINBOOKER, M.', 'PESScavolini PesaroScavolini Pesaro', 20, 75, 3.75, 4.44, 2001),
 (4, 5, 'EDNEY, TYUSEDNEY, T.', 'TREBenetton BasketBenetton Basket', 19, 72, 3.79, 5.0, 2001),
 (5, 6, 'HAWKINS, MICHAELHAWKINS, M.', 'WKSIdea SlaskIdea Slask', 14, 70, 5.0, 6.26, 2001),
 (6, 7, 'GINOBILI, EMANUELGINOBILI, E.', 'VIRKinder Virtus BolognaKinder Virtus Bologna', 22, 66, 3.0, 4.21, 2001),
 (7, 8, 'MC CANTS, CURTISMC CANTS, C.', 'CSKCSKA MoscowCSKA Moscow', 19, 65, 3.42, 4.61, 2001),
 (8, 9, 'FORD, ALPHONSOFORD, A.', 'OLYOlympiacosOlympiacos', 20, 64, 3.2, 3.63, 2001),
 (9, 9, 'BROWN, MARCUSBROWN, M.', 'EFSEfes PilsenEfes Pilsen', 19, 64, 3.37, 3.86, 2001),
 (10, 11, 'CELESTAND, JOHNCE